In [26]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [27]:
# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [28]:
# Batch size during training
batch_size = 128

# Number of training epochs
num_epochs = 50

# Learning rate for optimizers
lr = 0.0002

# Dimensions for generator
gen_input = 100
gen_output = 784

# Dimensions for Discriminator
dis_input = 784
dis_output = 1

In [2]:
## Preprocessing
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Reshape X_train into a 2d vector
x_train = x_train.reshape(x_train.shape[0], -1)

In [3]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_train.shape

(60000, 784)

## Generator

In [4]:
# Generator

def make_generator():
    model = Sequential()
    model.add(Dense(7*7*256, use_bias = False, input_shape=(100,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size
    
    model.add(Conv2DTranspose(128, (5, 5), strides = (1, 1), padding="same", use_bias = "false"))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(64, (5, 5), strides = (2, 2), padding="same", use_bias = "false"))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(1, (5, 5), strides = (2, 2), padding="same", use_bias = "false"))
    model.add(Activation("tanh"))
    assert model.output_shape == (None, 28, 28, 1)
    
    return model
    

In [36]:
generator = make_generator()

noise = tf.random.normal([1, 100])
generated_image = generator(noise)
# plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [37]:
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.1)

## Discriminator

In [30]:
## Discriminator
def make_discriminator():
    
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides = (2, 2), padding = "same", input_shape=[28, 28, 1], activation = lrelu))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, (5, 5), strides = (2, 2), padding = "same", activation = lrelu))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(1))
    
    return model

In [34]:
discriminator = make_discriminator()
decision = discriminator(generated_image)

In [38]:
## Loss Function
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [39]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [40]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [41]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)